In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup # https://realpython.com/beautiful-soup-web-scraper-python/
import time
from datetime import date

# GRAB ALL TICKERS INSIDE PRICE (MIN-MAX) RANGE

In [ ]:
pages = []
tickers_list = []
tickers_group_list = []

ticker_skip = 1900
price_min = 5.00
price_max = 25.00
valid_page_status = True

start_time = time.time()

while (valid_page_status):    
    
    url =   f"""
            https://www.marketwatch.com/tools/screener/stock?exchange=all&skip={ticker_skip}
            &orderbyfield=&direction=desc
            &visiblecolumns=Symbol,CompanyName,Price,NetChange,ChangePercent,Volume
            &pricemin={price_min}&pricemax={price_max}
            """ # must be inside while loop because {page_skip} won't update if outside of it...

    page = requests.get(url)
    soup_obj = soup(page.content, 'html.parser')
    ticker_skip += 25
    
    cells_found = soup_obj.find(class_ = "table__cell j-Symbol")

    if(cells_found):
        pages.append(url)
    else:
        valid_page_status = False
    
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Length of Pages[]: {len(pages)}")    
print(f"Elapsed Time: {elapsed_time}")

In [ ]:
for item in pages:
    page = requests.get(item)

    df = pd.read_html(page.content)

    tickers = df[0]
    tickers = tickers["Symbol"]
    
    for t in tickers:
        tickers_list.append(t)
        
    tickers_group_list.append(tickers)
    

import csv    
today = date.today()
date_format = today.strftime("%b-%d-%Y")
file_name = "tickers_" + date_format

with open(file_name, 'w') as f:
     
    write = csv.writer(f)
    write.writerows(tickers_group_list) 

# GRAB PRICE CHART TABLES FOR EVERY TICKER

In [ ]:
month = today.strftime("%B").lower()
year = today.strftime("%Y")
day = today.strftime("%d")
unix_time_yesterday = -1
unix_time_5years = -1

unix_converter_url = f"https://timeconverter.online/list/{year}/{month}"
page = requests.get(unix_converter_url)
soup_obj = soup(page.content, 'html.parser')
table_rows = soup_obj.find_all("td")

# Get the unix time stamp for today's date
for item in table_rows:
    date_td = item.text.split(" ")
    if(date_td[0] == day):
        unix_time_yesterday = item.find_previous().find_previous().text
        # find_previous() written twice, since having been read already, the iterator has moved
        
year = str((int(year) - 5))
unix_converter_url = f"https://timeconverter.online/list/{year}/{month}"
page = requests.get(unix_converter_url)
soup_obj = soup(page.content, 'html.parser')
table_rows = soup_obj.find_all("td") 

# Get the unix time stamp for date from 5 years ago
for item in table_rows:
    date_td = item.text.split(" ")
    if(date_td[0] == day):
        unix_time_5years = item.find_previous().find_previous().text
        
main_url = f"https://finance.yahoo.com/quote/{tickers_list[0]}?p={tickers_list[0]}"
        
base_url = f"https://finance.yahoo.com/quote/{tickers_list[0]}/history?p={tickers_list[0]}"

weekly_url =    f"""
                https://finance.yahoo.com/quote/{tickers_list[0]}/history?
                period1={unix_time_5years}&period2={unix_time_yesterday}
                &interval=1wk&filter=history&frequency=1wk
                &includeAdjustedClose=true    
                """

# https://waterfrontonline.files.wordpress.com/2022/10/delistingrules.pdf
compliance = "https://listingcenter.nasdaq.com/noncompliantcompanylist.aspx"

#https://stackoverflow.com/questions/76115408/web-scrapping-yahoo-finance
# credit goes to: https://stackoverflow.com/users/8743880/udhavsethi
page = requests.get(weekly_url, headers={'user-agent': 'custom'})  # added: headers={'user-agent': 'custom'}

df = pd.read_html(page.content)

soup_obj = soup(page.content, 'html.parser')
table_rows = soup_obj.find_all("td")

table_rows



# Stock Uprise Causes

[company] Reports First Quarter 2023 Financial Results
https://finance.yahoo.com/news/ainos-reports-first-quarter-2023-201500582.html



https://finance.yahoo.com/news/nasdaq-approves-180-day-extension-113000124.html
https://finance.yahoo.com/news/meiwu-technology-company-limited-received-130000250.html

In [2]:
import csv
import os

# compliance_list = []
# filepath = os.path.join("NasdaqNonComplianceIssuers.csv")
# with open(filepath, encoding='utf') as csvfile:
#     csvreader = csv.reader(csvfile, delimiter=",")
#     csv_header = next(csvreader)    # Date, Profit/Losses
#     for row in csvreader:
#         print(row)
#         compliance_list.append(row[1])  

# https://listingcenter.nasdaq.com/NonCompliantCompanyList.aspx
non_compliance_all_df = pd.read_csv ('NasdaqNonComplianceIssuers.csv')
bid_price_df = non_compliance_all_df.loc[non_compliance_all_df['Deficiency'] == "Bid Price"]

# define a new column
reduced_df = bid_price_df
reduced_df["Price"] = ""

reduced_df = bid_price_df[["Affected Issues", "Notification Date", "Price"]]
# only about half of the list is "bid price", so index and row values will not line up
reduced_df.reset_index(drop=True, inplace=True)


# Yahoo Finance only permits a limited amount of calls per hour, so break up the requests as needed...

for index, row in reduced_df.iterrows():
    
    if(index < 50):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text     

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\2628440690.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"] = ""
C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\2628440690.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text


In [3]:
for index, row in reduced_df.iterrows():
    
    if(index >= 50 and index < 100):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\1200540594.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text


In [4]:
for index, row in reduced_df.iterrows():
    
    if(index >= 100 and index < 150):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\230199425.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text


In [10]:
for index, row in reduced_df.iterrows():
    
    if(index >= 150 and index < 200):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\4143852784.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text
C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\4143852784.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found


In [14]:
for index, row in reduced_df.iterrows():
    
    if(index >= 200 and index < 250):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\91477333.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found
C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\91477333.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text


In [19]:
for index, row in reduced_df.iterrows():
    
    if(index >= 250 and index < 300):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\2577718552.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text


In [8]:
for index, row in reduced_df.iterrows():
    
    if(index >= 300 and index < 350):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\2881877909.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found
C:\Users\siloa\AppData\Local\Temp\ipykernel_14880\2881877909.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Price"][index] = price_found.text


In [ ]:
for index, row in reduced_df.iterrows():
    
    if(index >= 350 and index < 400):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

In [ ]:
for index, row in reduced_df.iterrows():
    
    if(index >= 400 and index < 450):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

In [ ]:
for index, row in reduced_df.iterrows():
    
    if(index >= 450 and index < 500):
        #grab the url to scrape
        url = f'https://finance.yahoo.com/quote/{row["Affected Issues"]}'
        page = requests.get(url, headers={'user-agent': 'custom'})
        soup_obj = soup(page.content, 'html.parser')
        # https://stackoverflow.com/questions/52816683/beautiful-soup-find-element-with-multiple-classes
        price_found = soup_obj.find(class_ = "Fw(b) Fz(36px) Mb(-4px) D(ib)")

        # 'NoneType' object has no attribute 'text'
        NoneType = type(None)
        if(type(price_found) == NoneType):
            reduced_df["Price"][index] = price_found
        else:
            reduced_df["Price"][index] = price_found.text  

In [22]:
sorted_df = reduced_df.sort_values(['Price'], ascending=[True])
sorted_df.to_csv("temp_list_sorted", sep='\t', encoding='utf-8')